In [28]:

import re
from sys import prefix
import pandas as pd
import time
import os
import shutil
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

class QF_NRA:
    count = 0
    index = 0
    def __init__(self, init=False, path='../../QF_NRA', solver='z3', execl='charts/base.xlsx'):
        self.path = path
        self.solver = solver
        self.data_path = 'results/' + solver
        self.records = pd.read_excel(execl, header=[0], index_col = [0])
        if init:
            self.records = self.records.drop(index=self.records.index)
            self.get_info(path)

    def get_info(self, path):
        folder_list = os.listdir(path)
        for folder in folder_list:
            new_path = os.path.join(path, folder)
            if os.path.isdir(new_path):
                self.get_info(new_path)
            else:
                with open(new_path) as f:
                    vars = 0
                    assertions = 0
                    status = 'none'
                    line = f.readline()
                    while line:
                        if line[:3] == '(de':
                            vars += 1
                        elif line[:3] == '(as':
                            assertions += 1
                        elif line[11:17] == 'status':
                            status = line.split(' ')[2][:-2]
                        line = f.readline()
                    # print(count, "/", total,": ", filename, "+", vars, "+", assertions)
                    self.count += 1
                    new_path = new_path[6:].replace('temp/', '')
                    category = new_path.split('/')[1]
                    # self.records = self.records.concat({'filename': new_path, 'category':category, 'vars':vars, 'assertions':assertions}, ignore_index=True, verify_integrity=True)
                    self.records.loc[len(self.records.index)] = [new_path, category, vars, assertions, status, '0', '0']

    def arrange(self):
        results_list = os.listdir(self.data_path)
        # total_num = len(results_list)
        index = 0   
        for folder in results_list:
            index += 1
            result_name = 'result_' + str(index)
            time_name = 'time_' + str(index)
            folder_path = os.path.join(self.data_path, folder)
            folder_list = os.listdir(folder_path)
            for filename in tqdm(folder_list, desc=folder_path):
                with open(os.path.join(folder_path, filename)) as f:
                    try:
                        if self.data_path == 'results/z3':
                            while f.readline():
                                result = f.readline()[:-1]
                                if result[:3] == 'run':
                                    result = 'timeout'
                                if result[:3] == 'tac':
                                    result = 'timeout'
                                    f.readline()
                                info = f.readline()[:-1]
                                name = info.split(' : ')[0]
                                timeout = info.split(' : ')[1][:-4]
                                self.records.loc[self.records['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                                if len(self.records.loc[self.records['filename'] == name]) == 0:
                                    print(name)
                        elif self.data_path == 'results/z3_UD':
                            while f.readline():
                                result = f.readline()[:-1]
                                if result[:3] == 'run':
                                    result = 'timeout'
                                if result[:3] == 'tac':
                                    result = 'timeout'
                                    f.readline()
                                if result[:3] == './s':
                                    result = 'fault'
                                info = f.readline()[:-1]
                                name = info.split(' : ')[0]
                                timeout = info.split(' : ')[1][:-4]
                                self.records.loc[self.records['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                                if len(self.records.loc[self.records['filename'] == name]) == 0:
                                    print(name)
                        elif self.data_path == 'results/cvc5':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                while info[:2] != 'QF' and info:
                                    result = info[:-1]
                                    info = f.readline()
                                if result[:3] == 'run':
                                    result = 'timeout'
                                name = info.split(' : ')[0]
                                timeout = info.split(' : ')[1][:-4]
                                self.records.loc[self.records['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                        elif self.data_path == 'results/yices2':
                            while f.readline():
                                result = f.readline()[:-1]
                                if result[:3] == 'run':
                                    result = 'dump'
                                info = f.readline()[:-1]
                                name = info.split(' : ')[0]
                                timeout = info.split(' : ')[1][:-4]
                                self.records.loc[self.records['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                        elif self.data_path == 'results/mathsat5':
                            while f.readline():
                                result = f.readline()[:-1]
                                if result[:3] == './s':
                                    result = 'timeout'
                                if result[:3] == 'tcm':
                                    result = 'timeout'
                                    info = f.readline()
                                    while info[:3] == 'tcm':
                                        info = f.readline()
                                if result[:3] == '(er':
                                    result = 'not support'
                                info = f.readline()[:-1]
                                name = info.split(' : ')[0]
                                timeout = info.split(' : ')[1][:-4]
                                self.records.loc[self.records['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                        elif self.data_path == 'results/smtrat':
                            while f.readline():
                                result = f.readline()[:-1]
                                if result[:3] == './s':
                                    result = 'timeout'
                                if result[:3] == 'QF_':
                                    info = result
                                    result = 'none'
                                else:
                                    info = f.readline()[:-1]
                                name = info.split(' : ')[0]
                                timeout = info.split(' : ')[1][:-4]
                                self.records.loc[self.records['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                                if len(self.records.loc[self.records['filename'] == name]) == 0:
                                    print(name)
                    except:
                        # print(filename, ' ', info, ' ', result)
                        pass
    
    def arrange_mbo(self, res='result', tim='time'):
        file_list = os.listdir(self.data_path)
        for filename in tqdm(file_list, desc=self.data_path):
            with open(os.path.join(self.data_path, filename)) as f:
                try:
                    while f.readline():
                        result = f.readline()[:-1]
                        if result[:3] == 'run':
                            result = 'dump'
                        info = f.readline()
                        if info[:2] == '(e':
                            result += 'error'
                            info = f.readline()
                        # QF_NRA/20161105-Sturm-MBO/mbo_E14E26.smt2
                        name = 'QF_NRA/20161105-Sturm-MBO/' + info.split(' : ')[0].split('/')[-1]
                        timeout = info.split(' : ')[1][:-4]
                        self.records.loc[self.records['filename'] == name, [res, tim]] = [result, timeout]
                except:
                    print(filename, ' ', info)
                         
    def save_to(self, filename):
        self.records.to_excel(filename)
    
    def show(self):
        folder_list = os.listdir(self.path)
        count = []
        solved = []
        dump = []
        timeout = []
        for folder in folder_list:
            count.append(len(self.records[self.records['category'] == folder]))
            solved.append(len(self.records[(self.records['category'] == folder) & (self.records['result'].str.contains('sat'))]))
            dump.append(len(self.records[(self.records['category'] == folder) & (self.records['result'] == ('dump'))]))
            timeout.append(len(self.records[(self.records['category'] == folder) & (self.records['result'] == ('timeout'))]))
        plt.figure(figsize=(18, 8))
        plt.bar(folder_list, solved, label="solved", color='green')
        plt.bar(folder_list, timeout, label="timeout", bottom=solved, color='orange')
        bot = [i + j for i, j in zip(solved, timeout)]
        plt.bar(folder_list, dump, label="dump", bottom=bot, color='red')
        # plt.bar(folder_list, count, label="total", bottom=timeout, color='gray')
        plt.legend(fontsize=11, frameon=False, loc='upper center')
        plt.xticks(rotation=270)
        plt.show()
    
    def get_timeout(self, source_path='../../QF_NRA', target_path='../../QF_NRA_timeout'):
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        if os.path.exists(target_path):
            shutil.rmtree(target_path)
        shutil.copytree(source_path, target_path)
        print('copy dir finished, begin deleting!')

    def delete(self, path, path_prefix, screen='result', screen_result='timeout'):
        screen_result = screen_result.split(',')
        folder_list = os.listdir(path)
        for folder in folder_list:
            new_path = os.path.join(path, folder)
            if os.path.isdir(new_path):
                self.delete(new_path)
            else:
                name = new_path.replace(path, path_prefix)
                # print(name)
                flag = True
                for result in screen_result:
                    if len(self.records[(self.records['filename'] == name) & (self.records[screen] == (result))]) == 1:
                        flag = False
                        break
                if flag:
                    os.remove(new_path)
                    
    def get_number(self, path):
        index = 0
        folder_list = os.listdir(path)
        for folder in folder_list:
            new_path = os.path.join(path, folder)
            if os.path.isdir(new_path):
                index += self.get_number(new_path)
            else:
                index += 1
        return index

In [29]:
a = QF_NRA(execl='charts/base.xlsx', solver='z3_UD')
a.arrange()
a.save_to('charts/result_z3_UD.xlsx')

results/z3_UD/z3_UD_1:   0%|          | 0/125 [00:00<?, ?it/s]

results/z3_UD/z3_10:   0%|          | 0/125 [00:00<?, ?it/s]

In [ ]:
a = QF_NRA(init=True)
a.records
a.save_to('charts/base.xlsx')

In [ ]:
a = QF_NRA(execl='charts/base.xlsx')
a.arrange()
a.save_to('charts/result_z3.xlsx')

In [ ]:
a = QF_NRA(execl='charts/base.xlsx', solver='cvc5')
a.arrange()
a.save_to('charts/result_cvc5.xlsx')

In [ ]:
a = QF_NRA(execl='charts/base.xlsx', solver='smtrat')
a.arrange()
a.save_to('charts/result_smtrat.xlsx')

In [7]:
a = QF_NRA(execl='charts/base.xlsx', solver='mathsat5')
a.arrange()
a.save_to('charts/result_mathsat5.xlsx')

results/mathsat5/mathsat5_8:   0%|          | 0/125 [00:00<?, ?it/s]

results/mathsat5/mathsat5_6:   0%|          | 0/125 [00:00<?, ?it/s]

results/mathsat5/mathsat5_10:   0%|          | 0/125 [00:00<?, ?it/s]

result_10.log      


results/mathsat5/mathsat5_2:   0%|          | 0/125 [00:00<?, ?it/s]

...

results/mathsat5/mathsat5_5:   0%|          | 0/125 [00:00<?, ?it/s]

results/mathsat5/mathsat5_1:   0%|          | 0/125 [00:00<?, ?it/s]

result_121.log   --------------------------------------------------
   QF_NRA/meti-tarski/sqrt/problem/13/sqrt-problem-13-chunk-0013.smt2 : 9 ms.
result_93.log   --------------------------------------------------
   QF_NRA/meti-tarski/polypaver/bench-sqrt-3d/polypaver-bench-sqrt-3d-chunk-0338.smt2 : 8 ms.
result_102.log   --------------------------------------------------
   QF_NRA/meti-tarski/sin/cos/sin-cos-346-b-chunk-0010.smt2 : 10 ms.
result_83.log   --------------------------------------------------
   QF_NRA/meti-tarski/exp/problem/10/2/exp-problem-10-2-chunk-0218.smt2 : 10 ms.
result_66.log   --------------------------------------------------
   QF_NRA/meti-tarski/Chua/1/VC2/L/Chua-1-VC2-L-chunk-0017.smt2 : 10 ms.
result_61.log   --------------------------------------------------
   QF_NRA/meti-tarski/cbrt/3/weak/cbrt-problem-3-weak-chunk-0021.smt2 : 11 ms.
result_112.log   --------------------------------------------------
   QF_NRA/meti-tarski/sqrt/1mcosq/7/sqrt-1mcosq-7-chun

results/mathsat5/mathsat5_7:   0%|          | 0/125 [00:00<?, ?it/s]

results/mathsat5/mathsat5_3:   0%|          | 0/125 [00:00<?, ?it/s]

In [ ]:
a = QF_NRA(execl='result_cvc5_3.xlsx', data_path='result_QF_NRA_part_new')
a.arrange()
a.save_to('result_cvc5_4.xlsx')

mbo部分

In [ ]:
a = QF_NRA(execl='base.xlsx', data_path='result_1')
a.arrange_mbo()
a.save_to('result_mbo_1.xlsx')

a = QF_NRA(execl='base.xlsx', data_path='result_2')
a.arrange_mbo()
a.save_to('result_mbo_2.xlsx')

a = QF_NRA(execl='base.xlsx', data_path='result_3')
a.arrange_mbo()
a.save_to('result_mbo_3.xlsx')

附加记录

In [ ]:
a = QF_NRA(execl='result_mbo_4.xlsx', data_path='result_4')
a.arrange_mbo(res='4_result', tim='4_time')
a.save_to('result_mbo_4.xlsx')

a = QF_NRA(execl='result_mbo_4.xlsx', data_path='result_5')
a.arrange_mbo(res='5_result', tim='5_time')
a.save_to('result_mbo_4.xlsx')

a = QF_NRA(execl='result_mbo_4.xlsx', data_path='result_control1')
a.arrange_mbo(res='control1_result', tim='control1_time')
a.save_to('result_mbo_4.xlsx')

a = QF_NRA(execl='result_mbo_4.xlsx', data_path='result_control2')
a.arrange_mbo(res='control2_result', tim='control2_time')
a.save_to('result_mbo_4.xlsx')

In [ ]:
a = QF_NRA(execl='result_mbo.xlsx', data_path='result')
a.arrange()
a.save_to('result_mbo_4.xlsx')

mbo难解部分

In [ ]:
a = QF_NRA(execl='result_mbo_4.xlsx', data_path='result')
a.get_timeout(source_path='../../QF_NRA/20161105-Sturm-MBO', target_path='../../QF_NRA_MBO')
a.delete(path='../../QF_NRA_MBO', screen='3_result', screen_result='timeout,unsaterror', path_prefix='QF_NRA/20161105-Sturm-MBO')
# a.save_to('result_mbo_4.xlsx')